In [181]:
import pandas as pd
df = pd.read_csv('US_Crime_DataSet_Original.csv')

C:\Users\patbr\AppData\Local\Temp\ipykernel_11204\2784183536.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('US_Crime_DataSet_Original.csv')


In [182]:
df.head()

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,1,FBI


In [183]:
len(df)

638454

In [184]:
# Clean whitespaces
df = df.apply(lambda col: col.str.strip() if col.dtypes == 'object' else col)

In [185]:
# Weapon Mapping
weapon_mapping = {
    'Blunt Object': 'Non-Firearm',
    'Strangulation': 'Non-Firearm',
    'Unknown': 'Unknown',
    'Rifle': 'Firearm',
    'Knife': 'Non-Firearm',
    'Firearm': 'Firearm',
    'Shotgun': 'Firearm',
    'Fall': 'Non-Firearm',
    'Handgun': 'Firearm',
    'Drowning': 'Non-Firearm',
    'Suffocation': 'Non-Firearm',
    'Explosives': 'Non-Firearm',
    'Fire': 'Non-Firearm',
    'Drugs': 'Non-Firearm',
    'Gun': 'Firearm',
    'Poison': 'Non-Firearm'
}
df['Weapon Category'] = df['Weapon'].replace(weapon_mapping)

In [186]:
df = df[df['Weapon Category'] != 'Unknown']

In [187]:
# Relationship Mappings
relationship_mapping = {
    'Family': ['Mother', 'Father', 'Stepdaughter', 'Stepfather',
               'Stepson', 'Stepmother', 'Daughter', 'Son', 'Brother', 'Sister', 
               'In-Law', 'Family'],
    'Lover': ['Wife', 'Husband', 'Girlfriend', 'Boyfriend', 'Boyfriend/Girlfriend', 'Ex-Husband', 'Ex-Wife', 'Common-Law Husband', 'Common-Law Wife'],
    'Acquaintance': ['Acquaintance', 'Neighbor', 'Friend', 'Employer', 'Employee'],
    'Stranger': ['Stranger']
}

# Create a reverse mapping dictionary
reverse_mapping = {relation: category for category, relations in relationship_mapping.items() for relation in relations}

# Map the categories to a new column
df['Relationship Category'] = df['Relationship'].map(reverse_mapping)
df = df.dropna(subset=['Relationship Category'])

In [188]:
df['Relationship Category'].unique()

array(['Acquaintance', 'Lover', 'Stranger', 'Family'], dtype=object)

In [189]:
df['Perpetrator Age'].unique()

array([nan, '21', '32', '28', '20', '61', '38', '18', '17', '53', '25',
       '84', '43', '30', '16', '47', '54', '22', '50', '42', '36', '24',
       '35', '60', '27', '19', '14', '23', '63', '81', '31', '51', '15',
       '0', '40', '33', '26', '41', '29', '34', '52', '37', '69', '44',
       '48', '55', '56', '45', '49', '65', '39', '59', '70', '46', '11',
       '58', '77', '67', '76', '57', '85', '13', '89', '71', '64', '62',
       '3', '80', '73', '75', '83', '72', '66', '86', '68', '74', '87',
       '78', '95', '6', '12', '88', '9', '90', '93', '10', '92', '79',
       '1', '82'], dtype=object)

In [190]:
df = df[df['Crime Solved'] == 'Yes']

In [53]:
len(df)

340375

In [93]:
df['Agency Type'].value_counts()

Agency Type
Municipal Police    252116
Sheriff              66147
County Police        11209
State Police          9313
Special Police        1415
Regional Police        145
Tribal Police           30
Name: count, dtype: int64

In [191]:
# less then 6000 rename to Other Police
df['Agency Type'] = df['Agency Type'].astype(str)
df['Agency Type'] = df['Agency Type'].replace('County Police', 'Other Police')
df['Agency Type'] = df['Agency Type'].replace('State Police', 'Other Police')
df['Agency Type'] = df['Agency Type'].replace('Special Police', 'Other Police')
df['Agency Type'] = df['Agency Type'].replace('Regional Police', 'Other Police')
df['Agency Type'] = df['Agency Type'].replace('Tribal Police', 'Other Police')

In [95]:
df['Agency Type'].value_counts()

Agency Type
Municipal Police    252116
Sheriff              66147
Other Police         22112
Name: count, dtype: int64

In [192]:
df = df[(df['Victim Race'] != 'Unknown') & (df['Victim Ethnicity'] != 'Unknown')]
df = df[(df['Perpetrator Race'] != 'Unknown') & (df['Perpetrator Ethnicity'] != 'Unknown')]

In [97]:
len(df)

155855

In [121]:
from scipy.stats import chi2_contingency

In [193]:
contingency_table = pd.crosstab(df['Victim Race'], df['Victim Ethnicity'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
# Display results
print("\nContingency Table:\n", contingency_table)
print("\nChi-square statistic:", chi2)
print("P-value", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:\n", expected)


Contingency Table:
 Victim Ethnicity               Hispanic  Not Hispanic
Victim Race                                          
Asian/Pacific Islander               36          3078
Black                               598         56325
Native American/Alaska Native         7          1434
White                             34536         59841

Chi-square statistic: 26923.29458447682
P-value 0.0
Degrees of freedom: 3
Expected frequencies:
 [[  702.84031953  2411.15968047]
 [12847.71339386 44075.28660614]
 [  325.2385679   1115.7614321 ]
 [21301.20771871 73075.79228129]]


In [194]:
# Interpretation
if p < 0.05:
    print("There is a significant association between the variables.")
else:
    print("There is no significant association between the variables.")

There is a significant association between the variables.


In [124]:
import numpy as np

In [195]:
# Calculate Cramer's V
n = contingency_table.to_numpy().sum() # total no. of observations
k, r = contingency_table.shape # dimensions of the table
cramers_v = np.sqrt(chi2 / (n * min(k - 1, r - 1)))

# Display results
print("Chi-square statistic:", chi2)
print("Cramer's V:", cramers_v)

# Interpretation of below:
## V < 0.1: Weak
## 0.1 ≤ V < 0.3: Moderate
## V > 0.3: Strong

# This sets the basis for imputing the missing values

Chi-square statistic: 26923.29458447682
Cramer's V: 0.4156269802231352


In [196]:
cr_vict_subset = df[['Victim Race','Victim Ethnicity']].copy()
cr_vict_subset.replace('Unknown', np.nan, inplace=True)

In [197]:
cr_perp_subset = df[['Perpetrator Race','Perpetrator Ethnicity']].copy()
cr_perp_subset.replace('Unknown', np.nan, inplace=True)

In [198]:
prob_vict_ethn_given_race = pd.crosstab(cr_vict_subset['Victim Race'], cr_vict_subset['Victim Ethnicity'], normalize='index')
prob_vict_race_given_ethnicity = pd.crosstab(cr_vict_subset['Victim Ethnicity'], cr_vict_subset['Victim Race'], normalize='columns')

In [199]:
prob_perp_ethn_given_race = pd.crosstab(cr_perp_subset['Perpetrator Race'], cr_perp_subset['Perpetrator Ethnicity'], normalize='index')
prob_perp_race_given_ethnicity = pd.crosstab(cr_perp_subset['Perpetrator Ethnicity'], cr_perp_subset['Perpetrator Race'], normalize='columns')

In [200]:
# Step 5: Impute missing values
## Function to impute Victim Ethnicity based on Victim Race
def impute_vict_ethnicity(row, prob_table):
    if pd.isnull(row['Victim Ethnicity']) and row ['Victim Race'] in prob_table.index:
        return np.random.choice(prob_table.columns, p=prob_table.loc[row['Victim Race']].values)
    return row['Victim Ethnicity']

## Function to impute Victim Race based on Victim Ethnicity
def impute_vict_race(row, prob_table):
    if pd.isnull(row['Victim Race']) and row ['Victim Ethnicity'] in prob_table.columns:
        return np.random.choice(prob_table.index, p=prob_table[row['Victim Ethnicity']].values)
    return row['Victim Race']

## Apply imputations
df['Victim Ethnicity'] = cr_vict_subset.apply(impute_vict_ethnicity, axis=1, prob_table=prob_vict_ethn_given_race)
df['Victim Race'] = cr_vict_subset.apply(impute_vict_race, axis=1, prob_table=prob_vict_race_given_ethnicity)

In [105]:
df['Victim Ethnicity'].unique()

array(['Not Hispanic', 'Hispanic'], dtype=object)

In [106]:
df['Victim Race'].unique()

array(['Native American/Alaska Native', 'Black', 'White',
       'Asian/Pacific Islander'], dtype=object)

In [201]:
# Step 5: Impute missing values
## Function to impute Victim Ethnicity based on Victim Race
def impute_perp_ethnicity(row, prob_table):
    if pd.isnull(row['Perpetrator Ethnicity']) and row ['Perpetrator Race'] in prob_table.index:
        return np.random.choice(prob_table.columns, p=prob_table.loc[row['Perpetrator Race']].values)
    return row['Perpetrator Ethnicity']

## Function to impute Victim Race based on Victim Ethnicity
def impute_perp_race(row, prob_table):
    if pd.isnull(row['Perpetrator Race']) and row ['Perpetrator Ethnicity'] in prob_table.columns:
        return np.random.choice(prob_table.index, p=prob_table[row['Perpetrator Ethnicity']].values)
    return row['Perpetrator Race']

## Apply imputations
df['Perpetrator Ethnicity'] = cr_perp_subset.apply(impute_perp_ethnicity, axis=1, prob_table=prob_perp_ethn_given_race)
df['Perpetrator Race'] = cr_perp_subset.apply(impute_perp_race, axis=1, prob_table=prob_perp_race_given_ethnicity)

In [132]:
df['Perpetrator Ethnicity'].unique()

array(['Not Hispanic', 'Hispanic'], dtype=object)

In [133]:
df['Perpetrator Race'].unique()

array(['Native American/Alaska Native', 'Black', 'White',
       'Asian/Pacific Islander'], dtype=object)

In [135]:
df.head()

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source,Weapon Category,Relationship Category
44,45,AKASP00,Alaska State Police,Other Police,Juneau,Alaska,1980,November,4,Manslaughter by Negligence,...,NaN,Native American/Alaska Native,Not Hispanic,Wife,Drowning,0,0,FBI,Non-Firearm,Lover
52,53,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,July,1,Murder or Manslaughter,...,NaN,Black,Not Hispanic,Acquaintance,Handgun,0,0,FBI,Firearm,Acquaintance
53,54,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,July,2,Murder or Manslaughter,...,NaN,White,Not Hispanic,Husband,Handgun,0,0,FBI,Firearm,Lover
54,55,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,July,3,Murder or Manslaughter,...,NaN,Black,Not Hispanic,Acquaintance,Handgun,0,1,FBI,Firearm,Acquaintance
58,59,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,September,41,Murder or Manslaughter,...,NaN,White,Not Hispanic,Father,Rifle,0,0,FBI,Firearm,Family


In [202]:
df = df[(df['Victim Age'] != 0) & (df['Victim Age'] <= 100)]

In [158]:
len(df)

153512

In [203]:
# State to Region
state_to_region = {
    # East Coast
    'Rhodes Island': 'Northeast',
    'District of Columbia': 'Northeast',
    'Maine': 'Northeast', 
    'New Hampshire': 'Northeast', 
    'Vermont': 'Northeast', 
    'Massachusetts': 'Northeast', 
    'Rhode Island': 'Northeast', 
    'Connecticut': 'Northeast',
    'New York': 'Northeast', 
    'New Jersey': 'Northeast', 
    'Pennsylvania': 'Northeast',
    'Delaware': 'South', 
    'Maryland': 'South', 
    'Virginia': 'South', 
    'North Carolina': 'South',
    'South Carolina': 'South', 
    'Georgia': 'South', 
    'Florida': 'South', 
    'Alabama': 'South',
    'Mississippi': 'South', 
    'Louisiana': 'South', 
    'Arkansas': 'South', 
    'Tennessee': 'South',
    'Kentucky': 'South', 
    'West Virginia': 'South', 
    'Oklahoma': 'South', 
    'Texas': 'South',
    'Ohio': 'Midwest', 
    'Indiana': 'Midwest', 
    'Illinois': 'Midwest', 
    'Iowa': 'Midwest', 
    'Missouri': 'Midwest', 
    'Michigan': 'Midwest', 
    'Wisconsin': 'Midwest', 
    'Minnesota': 'Midwest',
    'North Dakota': 'Midwest', 
    'South Dakota': 'Midwest', 
    'Nebraska': 'Midwest', 
    'Kansas': 'Midwest',
    'Montana': 'West', 
    'Wyoming': 'West', 
    'Idaho': 'West', 
    'Washington': 'West', 
    'Oregon': 'West',
    'California': 'West', 
    'Nevada': 'West', 
    'Utah': 'West', 
    'Colorado': 'West', 
    'Arizona': 'West',
    'New Mexico': 'West', 
    'Alaska': 'West', 
    'Hawaii': 'West'
}

# Apply state_to_region to the 'State' column (replace 'State' with your column name)
df['Region'] = df['State'].map(state_to_region)

In [204]:
# Season
def month_to_season(month):
    if month in ['December', 'January', 'February']:
        return 'Winter'
    elif month in ['March', 'April', 'May']:
        return 'Spring'
    elif month in ['June', 'July', 'August']:
        return 'Summer'
    elif month in ['September', 'October', 'November']:
        return 'Autumn'

df['Season'] = df['Month'].apply(month_to_season)

In [205]:
# Delete all that have Victim Sex Unknown
df = df[df['Victim Sex'] != 'Unknown']

In [206]:
df.to_csv('FULL_df_3_single_weapon.csv', index=False)

In [163]:
df.columns

Index(['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City',
       'State', 'Year', 'Month', 'Incident', 'Crime Type', 'Crime Solved',
       'Victim Sex', 'Victim Age', 'Victim Race', 'Victim Ethnicity',
       'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race',
       'Perpetrator Ethnicity', 'Relationship', 'Weapon', 'Victim Count',
       'Perpetrator Count', 'Record Source', 'Weapon Category',
       'Relationship Category', 'Region', 'Season'],
      dtype='object')

In [207]:
df.head()

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source,Weapon Category,Relationship Category,Region,Season
44,45,AKASP00,Alaska State Police,Other Police,Juneau,Alaska,1980,November,4,Manslaughter by Negligence,...,Not Hispanic,Wife,Drowning,0,0,FBI,Non-Firearm,Lover,West,Autumn
52,53,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,July,1,Murder or Manslaughter,...,Not Hispanic,Acquaintance,Handgun,0,0,FBI,Firearm,Acquaintance,South,Summer
53,54,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,July,2,Murder or Manslaughter,...,Not Hispanic,Husband,Handgun,0,0,FBI,Firearm,Lover,South,Summer
54,55,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,July,3,Murder or Manslaughter,...,Not Hispanic,Acquaintance,Handgun,0,1,FBI,Firearm,Acquaintance,South,Summer
58,59,AL00100,Jefferson,Sheriff,Jefferson,Alabama,1980,September,41,Murder or Manslaughter,...,Not Hispanic,Father,Rifle,0,0,FBI,Firearm,Family,South,Autumn


In [208]:
df = df.drop(columns=['Record ID', 'Agency Code', 'Agency Name', 'City', 'State', 'Year', 'Month', 'Incident','Crime Type'])

In [168]:
df.head()

,Agency Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source,Weapon Category,Relationship Category,Region,Season
44,Other Police,Yes,Female,26,Native American/Alaska Native,Not Hispanic,Male,NaN,Native American/Alaska Native,Not Hispanic,Wife,Drowning,0,0,FBI,Non-Firearm,Lover,West,Autumn
52,Sheriff,Yes,Male,23,Black,Not Hispanic,Male,NaN,Black,Not Hispanic,Acquaintance,Handgun,0,0,FBI,Firearm,Acquaintance,South,Summer
53,Sheriff,Yes,Male,42,White,Not Hispanic,Female,NaN,White,Not Hispanic,Husband,Handgun,0,0,FBI,Firearm,Lover,South,Summer
54,Sheriff,Yes,Male,33,Black,Not Hispanic,Male,NaN,Black,Not Hispanic,Acquaintance,Handgun,0,1,FBI,Firearm,Acquaintance,South,Summer
58,Sheriff,Yes,Male,46,White,Not Hispanic,Male,NaN,White,Not Hispanic,Father,Rifle,0,0,FBI,Firearm,Family,South,Autumn


In [209]:
df = df.drop(columns=['Crime Solved','Perpetrator Age','Relationship', 'Weapon', 'Victim Count', 'Perpetrator Count', 'Record Source', 'Victim Race', 'Perpetrator Race'])

In [210]:
df.head()

,Agency Type,Victim Sex,Victim Age,Victim Ethnicity,Perpetrator Sex,Perpetrator Ethnicity,Weapon Category,Relationship Category,Region,Season
44,Other Police,Female,26,Not Hispanic,Male,Not Hispanic,Non-Firearm,Lover,West,Autumn
52,Sheriff,Male,23,Not Hispanic,Male,Not Hispanic,Firearm,Acquaintance,South,Summer
53,Sheriff,Male,42,Not Hispanic,Female,Not Hispanic,Firearm,Lover,South,Summer
54,Sheriff,Male,33,Not Hispanic,Male,Not Hispanic,Firearm,Acquaintance,South,Summer
58,Sheriff,Male,46,Not Hispanic,Male,Not Hispanic,Firearm,Family,South,Autumn


In [211]:
df.to_csv('clean_df_3_single_weapon.csv', index=False)

In [212]:
len(df)

153511